In [ ]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    # model="ibm-granite/granite-3.0-8b-instruct",
    # model="ibm-granite/granite-8b-code-instruct-128k",
    model="ibm-granite/granite-20b-code-instruct-8k",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_length":300, "temperature":0.2},
)

In [ ]:
examples = [
{
    "question": "Add a styled header for my bookstore landing page",
    "context": "Gradient Background and Font Styling for Header",
    "output": """
import ipywidgets as widgets
from IPython.display import display

# Create a styled header
header = widgets.HTML(value="""
    <div style='background: linear-gradient(to right, #6a11cb, #2575fc); padding: 20px; border-radius: 8px;'>
        <h1 style='color: white; text-align: center; font-family: Arial, sans-serif;'>Welcome to Reader's Verse</h1>
    </div>
""")

display(header)
"""
},
{
    "question": "Enhance my bookstore catalog with hover effects",
    "context": "Interactive Book Tiles with Shadows and Hover Animation",
    "output": """
import ipywidgets as widgets
from IPython.display import display

# Create book tiles with hover effects
book_tiles = [
    widgets.HTML(value="""
        <div style='background-color: #ffffff; padding: 20px; margin: 15px; border-radius: 8px; transition: transform 0.2s; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);'
             onmouseover="this.style.transform='scale(1.05)';"
             onmouseout="this.style.transform='scale(1.0)';">
            <h2 style='color: #444;'>The Great Gatsby</h2>
            <p style='color: #555;'><b>Author:</b> F. Scott Fitzgerald</p>
            <p style='color: #555;'><b>Price:</b> $10.00</p>
        </div>
    """),
    widgets.HTML(value="""
        <div style='background-color: #ffffff; padding: 20px; margin: 15px; border-radius: 8px; transition: transform 0.2s; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);'
             onmouseover="this.style.transform='scale(1.05)';"
             onmouseout="this.style.transform='scale(1.0)';">
            <h2 style='color: #444;'>Pride and Prejudice</h2>
            <p style='color: #555;'><b>Author:</b> Jane Austen</p>
            <p style='color: #555;'><b>Price:</b> $15.00</p>
        </div>
    """),
]

container = widgets.VBox(book_tiles)
display(container)
"""
},
{
    "question": "Add a footer with contact details for the bookstore",
    "context": "Footer with Solid Background and Contact Details",
    "output": """
import ipywidgets as widgets
from IPython.display import display

# Create a footer
footer = widgets.HTML(value="""
    <div style='background-color: #222; color: white; padding: 15px; text-align: center;'>
        <p>Contact Us: support@readersverse.com | Call: +1 800 123 4567</p>
        <p>Follow us on <a href='#' style='color: #1e90ff;'>Twitter</a> and <a href='#' style='color: #1e90ff;'>Facebook</a></p>
    </div>
""")

display(footer)
"""
},
{
    "question": "Enhance catalog tiles with alternating background colors",
    "context": "Alternating Background Colors for Visual Clarity",
    "output": """
import ipywidgets as widgets
from IPython.display import display

# Define book tiles with alternating background colors
book_titles = ["Book Title 1", "Book Title 2", "Book Title 3", "Book Title 4"]
book_tiles = []
for i, title in enumerate(book_titles):
    background = "#f5f5f5" if i % 2 == 0 else "#eaeaea"
    book_tiles.append(widgets.HTML(value=f"""
        <div style='background-color: {background}; padding: 15px; margin: 10px; border-radius: 8px;'>
            <h2 style='color: #333;'>{title}</h2>
            <p style='color: #555;'>Author: Author Name {i+1}</p>
            <p style='color: #555;'>Price: ${(i+1)*10}.00</p>
        </div>
    """))

container = widgets.VBox(book_tiles)
display(container)
"""
}
]

In [ ]:
def fewshot_prompt(context, question, book_titles, examples):
    """
    Creates a few-shot prompt for the model, where the model leverages examples to improve accuracy.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question or task for the model to perform
    - book_titles: list, list of book titles to include in the prompt
    - examples: list of dicts, each containing 'question', 'context', and 'output' as keys to provide examples
    Returns:
    - str, the formatted few-shot prompt
    """

    # Format the book titles
    titles = ", ".join(book_titles)

    # Format the examples for the prompt
    formatted_examples = "\n\n".join(
        f"""
        Example {i+1}:
        User Question: {example['question']}
        Context: {example['context']}
        Model Output: {example['output']}
        """
        for i, example in enumerate(examples)
    )

    # Construct the few-shot prompt
    prompt = f"""
    You are an experienced programmer with 15 years of experience writing full-stack applications.
    Your task is to generate high-quality Python code for a Jupyter Notebook using ipywidgets UI components
    based on the provided context and user question.

    Here are some examples of similar tasks you have completed successfully:
    {formatted_examples}

    Now, using these examples as a reference, generate code for the following task:
    Context: {context}
    User Question: {question}
    Include descriptions of elements from the book titles: {titles}
    Ensure that:
    - The code is well-structured and uses appropriate styling, coloring, and formatting for UI elements.
    - The output code is optimized and does not exceed 300 tokens.
    - Output only the Python code.
    """
    return prompt


In [ ]:
# Function to get answer using few-shot prompting
def get_answer_using_fewshot(context, question, book_titles):
    """
    Generates the response from the model based on a few-shot prompt.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question for the model to answer
    - book_titles: list, list of book titles to include in the prompt

    Returns:
    - str, the generated result from the model
    """
    prompt = fewshot_prompt(context, question, book_titles)
    result = model.invoke(prompt)

    return result

In [ ]:
# Example usage
context = "Design and develop an online bookstore UI components with minimalistic theme."
question = "Create the landing page for users visiting my bookstore. The landing page should display a header `Reader's Online Store`, a welcome message `Welcome to Reader's Verse` along with a catalog of books titles as title. Vertically align the sections."
book_titles = ["The Great Gatsby", "Pride and Prejudice", "The Hobbit", "The Lord of the Rings", "Animal Farm", "Brave New World"]
# Generate and display the UI code for the landing page
result = get_answer_using_fewshot(context, question, book_titles)
print(f"Generated Code:\n{result}")